In [1]:
import spacy;
from spacy import displacy;
from diaparser.parsers import Parser;
import os;


parser = Parser.load('models_ptbr/Diaparser/diaparser.model');
nlp = spacy.load("pt_core_news_sm")

Some weights of the model checkpoint at pucpr/biobertpt-all were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [2]:
def parseSpacyTokens(doc):
   tokens = [];
   for token in doc:
       tokens.append(token.text);
   dataset = parser.predict(tokens, text='en');
   return dataset;

def updateSpacyParser(doc, dataset):
    sent = dataset.sentences[0];
    for token in doc:
        head_idx = sent.arcs[token.i];
        token.dep_ = sent.rels[token.i];
        if head_idx == 0:
           token.head = token;
           token.dep_ = 'ROOT';
        else:
           token.head = doc[head_idx-1];
    return doc;

def saveList(listToSave, fname):
    with open(fname, 'w') as f:
         for item in listToSave:
           f.write("%s\n" % item)

def svg2PNG(doc, outFile, style='dep', linux = True):
   from pathlib import Path;
   svgFile = os.path.splitext(outFile)[0] + '.svg'; 

   svg = spacy.displacy.render(doc, style=style, jupyter=False);
   output_path = Path(svgFile);
   output_path.open("w", encoding="utf-8").write(svg);
   pngfile = os.path.splitext(outFile)[0] + '.png';
   
   if linux:
      os.system('inkscape -e %s %s' % (pngfile, svgFile)); 
   else: # windows
      cmd = '"C:\\Program Files\\Inkscape\\bin\\inkscape.com" -o "%s" %s' % (outFile, svgFile);
      bat = [cmd];
      saveList(bat, 'temp.bat');
      os.system('temp.bat');
    

In [3]:
texto = 'Há uma imagem nodular mediastinal, adjacente à veia cava superior / átrio direito, em contiguidade com a massa do lobo superior direito, medindo 1,5 x 1,5 cm, que pode representar linfonodomegalia ou extensão tumoral';

In [4]:
# uso o spacy para gerar os tokens e com isso garantir a volta do algoritmo
doc = nlp(texto);

# com os tokens gerados pelo spacy, gero o gráfico de dependencias com o diaparser
dataset = parseSpacyTokens(doc);
# trago a informação gerada pelo diaparser de volta para o spacy
doc = updateSpacyParser(doc, dataset);
# mostro o gráfico de dependencias
spacy.displacy.render(doc, style='dep');  
svg2PNG(doc, 'diaparser.svg');

In [5]:
docspacy = nlp(texto);
spacy.displacy.render(docspacy, style='dep');  
svg2PNG(docspacy, 'spacy.svg');